In [33]:
import pickle

# Read binary pickle file
with open("results_ctrlAV.pkl", "rb") as f:
    data = pickle.load(f)

In [34]:
import pandas as pd

df_list = []

for participant, roi_dict in data.items():
    for roi, model_dict in roi_dict.items():
        # convert each model: array to a column + keep its index
        temp = pd.DataFrame(model_dict)
        
        # Reset index so the row index = timepoint
        temp = temp.reset_index().rename(columns={'index': 'index'})
        # "index" now goes from 0 .. 1576
        
        # Melt to long format
        temp = temp.melt(id_vars="index", var_name="model", value_name="value")
        
        # Add metadata
        temp["participant"] = participant
        temp["roi"] = roi
        
        df_list.append(temp)

df = pd.concat(df_list, ignore_index=True)



In [35]:
df

,index,model,value,participant,roi
0,0,Llama_layers_layers7-11,-0.074724,33,2
1,1,Llama_layers_layers7-11,0.056496,33,2
2,2,Llama_layers_layers7-11,-0.128081,33,2
3,3,Llama_layers_layers7-11,0.130229,33,2
4,4,Llama_layers_layers7-11,-0.071431,33,2
...,...,...,...,...,...
2561043,1572,binder_concreteness,-0.029869,32,face
2561044,1573,binder_concreteness,0.012246,32,face
2561045,1574,binder_concreteness,-0.092258,32,face
2561046,1575,binder_concreteness,-0.035142,32,face


In [36]:
blind = {33, 35, 36, 38, 39, 41, 42, 43, 53}
ctrlA = {3, 4, 5, 6, 7, 8, 9, 10, 11, 27}
ctrlAV = {12, 13, 14, 15, 16, 17, 18, 19, 22, 32}

participant_to_group = {}

for p in blind:
    participant_to_group[p] = "blind"
for p in ctrlA:
    participant_to_group[p] = "ctrlA"
for p in ctrlAV:
    participant_to_group[p] = "ctrlAV"

df["participant_group"] = df["participant"].map(participant_to_group)


In [37]:
df

,index,model,value,participant,roi,participant_group
0,0,Llama_layers_layers7-11,-0.074724,33,2,blind
1,1,Llama_layers_layers7-11,0.056496,33,2,blind
2,2,Llama_layers_layers7-11,-0.128081,33,2,blind
3,3,Llama_layers_layers7-11,0.130229,33,2,blind
4,4,Llama_layers_layers7-11,-0.071431,33,2,blind
...,...,...,...,...,...,...
2561043,1572,binder_concreteness,-0.029869,32,face,ctrlAV
2561044,1573,binder_concreteness,0.012246,32,face,ctrlAV
2561045,1574,binder_concreteness,-0.092258,32,face,ctrlAV
2561046,1575,binder_concreteness,-0.035142,32,face,ctrlAV


In [38]:
# Read binary pickle file
with open("qwentext/results_ctrlAV.pkl", "rb") as f:
    new_data = pickle.load(f)

In [39]:
exclude_models = {"binder_abstractness", "binder_concreteness"}

def convert_nested_dict_to_df(data_dict, participant_to_group):
    df_list = []
    
    for participant, roi_dict in data_dict.items():
        for roi, model_dict in roi_dict.items():

        
            filtered = {m: arr for m, arr in model_dict.items() if m not in exclude_models}

            if not filtered:
                continue

            temp = (
                pd.DataFrame(filtered)
                .reset_index()
                .melt(id_vars="index", var_name="model", value_name="value")
            )

            temp["participant"] = participant
            temp["roi"] = roi
            temp["participant_group"] = participant_to_group[participant]

            df_list.append(temp)
    return pd.concat(df_list, ignore_index=True)

dfnew = convert_nested_dict_to_df(new_data, participant_to_group)



In [40]:
df_combined = pd.concat([df, dfnew], ignore_index=True)


In [41]:
df_combined

,index,model,value,participant,roi,participant_group
0,0,Llama_layers_layers7-11,-0.074724,33,2,blind
1,1,Llama_layers_layers7-11,0.056496,33,2,blind
2,2,Llama_layers_layers7-11,-0.128081,33,2,blind
3,3,Llama_layers_layers7-11,0.130229,33,2,blind
4,4,Llama_layers_layers7-11,-0.071431,33,2,blind
...,...,...,...,...,...,...
2926907,1572,qwen-text_layers16-20_conv,0.041527,32,face,ctrlAV
2926908,1573,qwen-text_layers16-20_conv,0.063453,32,face,ctrlAV
2926909,1574,qwen-text_layers16-20_conv,0.087940,32,face,ctrlAV
2926910,1575,qwen-text_layers16-20_conv,-0.029212,32,face,ctrlAV


In [42]:
with open("semantic/results_semantic.pkl", "rb") as f:
    semanticresults = pickle.load(f)

print(semanticresults)

{('binder_concreteness', 'Llama_layers_layers7-11'): array([-0.029333  , -0.03696641,  0.03942497, ...,  0.00633964,
        0.00707369,  0.00748555], shape=(1577,)), ('binder_concreteness', 'CLIPmultilingualmulti_layers11-15'): array([ 0.11705375,  0.1144265 ,  0.11454238, ..., -0.05413075,
       -0.04792858, -0.04692644], shape=(1577,)), ('binder_concreteness', 'qwen-text_layers16-20_conv'): array([ 0.06959846,  0.10228053,  0.06608655, ...,  0.10551191,
        0.09112249, -0.07064948], shape=(1577,)), ('binder_concreteness', 'qwen-audiovideotext_layers16-20'): array([ 0.03039857,  0.04032052,  0.05169334, ...,  0.11369727,
        0.1018167 , -0.04168547], shape=(1577,)), ('binder_concreteness', 'CLIPmultilingualtext_layers11-15'): array([ 0.13917391,  0.12873833,  0.12423615, ..., -0.02152618,
       -0.01561056, -0.01451901], shape=(1577,)), ('binder_concreteness', 'XLM-roberta_layers11-15'): array([0.08773513, 0.13049882, 0.03640405, ..., 0.06752426, 0.06714298,
       0.066927

In [43]:
import numpy as np

corr_conc = {
    model: arr
    for (binder, model), arr in semanticresults.items()
    if binder == 'binder_concreteness'
}

corr_abs = {
    model: arr
    for (binder, model), arr in semanticresults.items()
    if binder == 'binder_abstractness'
}

df_combined["corr_concreteness"] = np.nan
df_combined["corr_abstractness"] = np.nan

for model_name, arr in corr_conc.items():
    mask = df_combined["model"] == model_name
    df_combined.loc[mask, "corr_concreteness"] = arr[df_combined.loc[mask, "index"].values]
    
for model_name, arr in corr_abs.items():
    mask = df_combined["model"] == model_name
    df_combined.loc[mask, "corr_abstractness"] = arr[df_combined.loc[mask, "index"].values]


In [44]:
df_combined

,index,model,value,participant,roi,participant_group,corr_concreteness,corr_abstractness
0,0,Llama_layers_layers7-11,-0.074724,33,2,blind,-0.029333,0.030983
1,1,Llama_layers_layers7-11,0.056496,33,2,blind,-0.036966,-0.029370
2,2,Llama_layers_layers7-11,-0.128081,33,2,blind,0.039425,0.005320
3,3,Llama_layers_layers7-11,0.130229,33,2,blind,0.046623,0.064804
4,4,Llama_layers_layers7-11,-0.071431,33,2,blind,0.057476,0.072272
...,...,...,...,...,...,...,...,...
2926907,1572,qwen-text_layers16-20_conv,0.041527,32,face,ctrlAV,0.133422,0.124665
2926908,1573,qwen-text_layers16-20_conv,0.063453,32,face,ctrlAV,0.124213,0.090562
2926909,1574,qwen-text_layers16-20_conv,0.087940,32,face,ctrlAV,0.105512,0.048710
2926910,1575,qwen-text_layers16-20_conv,-0.029212,32,face,ctrlAV,0.091122,0.026334


In [45]:
# Create the column
df_combined["roi_type"] = df_combined["roi"].apply(
    lambda x: "language" if isinstance(x, (int, float)) else "visual"
)

# Move roi_type right after roi
roi_index = df_combined.columns.get_loc("roi")
df_combined.insert(roi_index + 1, "roi_type", df_combined.pop("roi_type"))

In [46]:
run_start_indices = [0, 267, 492, 812, 1137, 1373]
df_combined["run"] = np.searchsorted(run_start_indices, df_combined["index"], side="right")

In [47]:
df_combined.to_csv("results.csv", sep=";", index=False)